In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython import display

Lecture1_Physics_Informed_Neural_Networks.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb

# Lectures on Machine Learning for Strong Gravity
## Lecture 1: Physics-Informed Neural Networks

To test it, simply press Ctrl+Enter sequentially in each cell, or click on the small icons on the left with the "play" symbol.

<br>

<a href="https://colab.research.google.com/github/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### In this lecture, you will learn:
1. How to use PINNs to solve differential equations, including ODEs, PDEs and eigenvalue problems.<br>Original paper:<br>

 - Maziar Raissi, Paris Perdikaris, George Em Karniadakis, <i>Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations</i>, Journal of Computational Physics, 378, 686-707, 2019. https://arxiv.org/abs/1711.10561
 
2. How these can be used, for instance, for the computation of QNMs.<br>Original paper:<br>

 - Raimon Luna, Juan Calderón Bustillo, Juan José Seoane Martínez, Alejandro Torres-Forné, José A. Font, <i>Solving the Teukolsky equation with physics-informed neural networks
</i>, Phys.Rev.D 107 (2023) 6, 064025, 2023. https://arxiv.org/abs/2212.06103

<br>
"""



In [ ]:
def gradients(outputs, inputs, order = 1):
    if order == 1:
        return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)[0]
    elif order > 1:
        return gradients(gradients(outputs, inputs, 1), inputs, order - 1) # Recursively take gradients
    else:
        return outputs

def generate_2Dgrid(minimum1, maximum1, minimum2, maximum2, N):
    grid1 = np.linspace(minimum1, maximum1, N, dtype = np.float32)
    grid2 = np.linspace(minimum2, maximum2, N, dtype = np.float32)
    x0, y0 = np.meshgrid(grid1, grid2)
    x = torch.tensor(x0.reshape(N**2), requires_grad = True)
    y = torch.tensor(y0.reshape(N**2), requires_grad = True)
    return x, y

def plot_form(x, y, z, N):
    return map(lambda t: t.reshape(N,N).cpu().detach().numpy(), (x, y, z))


"""# 1. Automatic differentiation and optimization"""

In [ ]:
x = torch.tensor(np.linspace(-2*np.pi, 2*np.pi, 100, dtype = np.float32)).reshape(100, 1) # list of xvalues to evaluate on
x.requires_grad = True

y = torch.sin(x) # sin(x)

dy_dx = gradients(y, x)
plt.plot(x.detach().numpy(), y.detach().numpy())
plt.plot(x.detach().numpy(), dy_dx.detach().numpy())

In [ ]:
x = torch.tensor([1.5], requires_grad = True)
y = torch.tensor([0.5], requires_grad = True)

track = []

#ADAM optimization algorithm 
#https://towardsdatascience.com/complete-guide-to-adam-optimization-1e5f29532c3d#:~:text=1.-,Definition%20of%20Adam%20Optimization,memory%20requirement%E2%80%9D%20%5B2%5D.
optimizer = optim.Adam([x, y], lr=0.1)

# Use ADAM to traverse the space defined by exp(-(x + y)**2 - y**2)
for i in range(100):
    optimizer.zero_grad()

    z = 1 - torch.exp( -(x + y)**2 - y**2  )
    z.backward()

    optimizer.step()
    track.append([x.item(), y.item()])

track = np.array(track)

x0, y0 = np.meshgrid(np.linspace(-2,2,100), np.linspace(-2,2,100))
plt.contourf(x0, y0, 1 - np.exp( -(x0 + y0)**2 - y0**2  ))
plt.scatter(track[:, 0], track[:, 1], color = 'red', marker='x')

## Challenges!
 - Try computing derivatives of other functions
 - Try changing the learning rate. What happens if it gets too small? Or too big?

In [ ]:
"""
import torch
import scipy.special as sp
x = torch.tensor(np.linspace(-2*np.pi, 2*np.pi, 100, dtype = np.float32)).reshape(100, 1) # list of xvalues to evaluate on
x.requires_grad = True

y =  sp.ellipk(x)# sin(x)
dy_dx = torch.autograd.grad(y, x, torch.ones_like(y), create_graph=True)[0]
"""

In [ ]:
x = torch.tensor(np.linspace(-2*np.pi, 2*np.pi, 100, dtype = np.float32)).reshape(100, 1) # list of xvalues to evaluate on
x.requires_grad = True

y = x**2# sin(x)

dy_dx = gradients(y, x)
plt.plot(x.detach().numpy(), y.detach().numpy())
plt.plot(x.detach().numpy(), dy_dx.detach().numpy())

# 2. Solving a system of ODEs: Sine and Cosine

Here we solve the system of ODEs

$$s'(x) = c(x),$$
$$c'(x) = -s(x),$$

with the initial conditions

$$s(0) = 0, \;c(0) = 1.$$

We impose the initial conditions by weak enforcement.

<br>

In [ ]:
# The ODE solution will be modelled as a NN with 1 hidden layer of 40 nodes 
# and output 2 values
class ODE(nn.Module):
    def __init__(self):
        super(ODE, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(1, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 2),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean = 0, std = 0.1)
                nn.init.constant_(m.bias, val = 0.0)
                
    def forward(self, x):
        return self.net(x)

# The loss function that will be used to train the NN will be constructed
# out of the DE
class ODELoss(nn.Module):
    def __init__(self, ode):
        super(ODELoss, self).__init__()
        self.ode = ode
        
    def forward(self, x):
        
        funcs  = self.ode(x)
        
        c, s = map(lambda i:  funcs[:,[i]], range(2))
        dc = gradients(c, x)
        ds = gradients(s, x)
        
        zero_vals = self.ode(torch.zeros(1,1))

        # loss function associated with satisfying DE        
        eq_loss = torch.mean((dc + s)**2 + (ds - c)**2)

        #loss function that constrains Initial Conditions
        ic_loss = (zero_vals[0,0] - 1)**2 + zero_vals[0,1]**2
        
        return eq_loss, ic_loss


In [ ]:
ode = ODE()
odeloss = ODELoss(ode)
loss_hist = []

optimizer = optim.Adam(ode.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=(0.05)**(1/4000))

x = torch.tensor(np.linspace(0, 10, 100, dtype = np.float32)).reshape(100, 1)
x.requires_grad = True

################## Training and Plotting ##################

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 4))
font = {'size'   : 19}
plt.rc('font', **font)

for epoch in range(4000):
    try:
        optimizer.zero_grad()

        eq_loss, ic_loss = odeloss(x)
        loss = eq_loss + ic_loss
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        loss_hist.append(loss.item())
        
        if epoch % 100 == 0: #plot every 100 epochs
            
            ax1.cla()
            ax1.set_xlabel('epoch')
            ax1.set_ylabel('loss')
            ax1.set_yscale('log')
            ax1.plot(loss_hist)

            ax2.cla()
            ax2.set_xlabel('x')
            ax2.set_ylabel('y')
            ax2.plot(x.cpu().detach().numpy(), ode(x)[:, 0].cpu().detach().numpy())
            ax2.plot(x.cpu().detach().numpy(), ode(x)[:, 1].cpu().detach().numpy())

            display.display(plt.gcf())
            display.clear_output(wait=True)
            plt.tight_layout()
            
    except KeyboardInterrupt:
        break

## Challenge!

 - Try imposing the conditions at $x = 2\pi$ instead of at $x = 0$. Does it affect the performance?

<br>

In [ ]:
class ODELoss2(nn.Module):
    def __init__(self, ode):
        super(ODELoss2, self).__init__()
        self.ode = ode
        
    def forward(self, x):
        
        funcs  = self.ode(x)
        
        c, s = map(lambda i:  funcs[:,[i]], range(2))
        dc = gradients(c, x)
        ds = gradients(s, x)
        
        twopi_vals = self.ode(torch.asarray([[2*np.pi]]))

        # loss function associated with satisfying DE        
        eq_loss = torch.mean((dc + s)**2 + (ds - c)**2)

        #loss function that constrains Initial Conditions
        ic_loss = (twopi_vals[0,0] - 1)**2 + twopi_vals[0,1]**2
        
        return eq_loss, ic_loss
    


In [ ]:
ode = ODE()
odeloss = ODELoss2(ode)
loss_hist = []

optimizer = optim.Adam(ode.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=(0.05)**(1/4000))

x = torch.tensor(np.linspace(0, 10, 100, dtype = np.float32)).reshape(100, 1)
x.requires_grad = True

################## Training and Plotting ##################

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 4))
font = {'size'   : 19}
plt.rc('font', **font)

for epoch in range(4000):
    try:
        optimizer.zero_grad()

        eq_loss, ic_loss = odeloss(x)
        loss = eq_loss + ic_loss
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        loss_hist.append(loss.item())
        
        if epoch % 100 == 0: #plot every 100 epochs
            
            ax1.cla()
            ax1.set_xlabel('epoch')
            ax1.set_ylabel('loss')
            ax1.set_yscale('log')
            ax1.plot(loss_hist)

            ax2.cla()
            ax2.set_xlabel('x')
            ax2.set_ylabel('y')
            ax2.plot(x.cpu().detach().numpy(), ode(x)[:, 0].cpu().detach().numpy())
            ax2.plot(x.cpu().detach().numpy(), ode(x)[:, 1].cpu().detach().numpy())

            display.display(plt.gcf())
            display.clear_output(wait=True)
            plt.tight_layout()
            
    except KeyboardInterrupt:
        break

# ASIDE: Solving the Inviscid Burgers equation

Here we solve the inviscid Burgers Equation

$$\partial_t u(t,x) = u(t,x) \partial_x u(t,x),$$

with the initial conditions

$$u(0,x) = f(x) = \begin{cases}1 && \text{if }x < 0\\1-x && \text{if }0\leq x \leq 1\\0 && \text{if }1< x \\\end{cases}$$

We impose the initial conditions by weak enforcement.

<br>

In [ ]:
def g(x):
    if x < -1:
        return 0
    elif x < 0:
        return x+1
    elif x < 1:
        return 1-x
    else:
        return 0
    
# The ODE solution will be modelled as a NN with 1 hidden layer of 40 nodes 
# and output 2 values
class InviscidBurgers(nn.Module):
    def __init__(self):
        super(InviscidBurgers, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 40),
            nn.Tanh(),
            #nn.Linear(40, 40),
            #nn.Tanh(),
            nn.Linear(40, 1),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean = 0, std = 0.1)
                nn.init.constant_(m.bias, val = 0.0)

    def hard_enforcement(self, t, x, u):
        return torch.mul(-(x**2-4), torch.mul(torch.exp(t)-1, u)) + x/torch.abs(x)#+ torch.asarray(list(map(g, x))), u)
            
    def forward(self, t, x):
        tx = torch.stack((t,x), 1)
        #bdry = (abs(x) - 1).reshape(len(x), 1)
        #return self.net(tx) 
        outnet = self.net(tx)
        return torch.mul((((torch.exp(2*t)-1)).reshape(len(t),1)), outnet) + (torch.asarray(list(map(g, x)))).reshape(len(t),1)

# The loss function that will be used to train the NN will be constructed
# out of the DE

class InviscidBurgersLoss(nn.Module):
    def __init__(self, approx_sol):
        super(InviscidBurgersLoss, self).__init__()
        self.approx_sol = approx_sol
        
    def forward(self, t, x):
        u   =  self.approx_sol(t, x)
        u_t = gradients(u, t, 1)
        u_x = gradients(u, x, 1)

        eq = u_t +u* u_x

        # loss function associated with satisfying DE        
        eq_loss = abs(eq)

        return torch.max(eq_loss)

In [ ]:
# The ODE solution will be modelled as a NN with 1 hidden layer of 40 nodes 
# and output 2 values
class InviscidBurgers(nn.Module):
    def __init__(self):
        super(InviscidBurgers, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 1),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean = 0, std = 0.1)
                nn.init.constant_(m.bias, val = 0.0)
                
    def forward(self, t, x):
        tx = torch.stack((t,x), 1)
        #bdry = (abs(x) - 1).reshape(len(x), 1)
        return self.net(tx) #* bdry

# The loss function that will be used to train the NN will be constructed
# out of the DE
def g(x):
    if x < -1:
        return 0
    elif x < 0:
        return x+1
    elif x < 1:
        return 1-x
    else:
        return 0


resolution = 1000
class InviscidBurgersLoss(nn.Module):
    def __init__(self, approx_sol):
        super(InviscidBurgersLoss, self).__init__()
        self.approx_sol = approx_sol
        
    def forward(self, t, x):

       # global test1, test2, test3, test4
      
        u   =  self.approx_sol(t, x).squeeze()
        u_t = gradients(u, t)
        u_x = gradients(u, x)

        eq = u_t + u* u_x

        # loss function associated with satisfying DE        
        eq_loss = (eq)**2



        #define the initial values of t,x
        tinitial=torch.zeros(resolution)
        xinitial=torch.as_tensor([x[resolution*i] for i in range(resolution)])

        #pde(torch.zeros(30),xvalues)-torch.as_tensor(list(map(g, xvalues)))

        ic = torch.flatten(self.approx_sol(tinitial,xinitial) )#self.approx_sol(torch.zeros(x.detach().numpy().size), x)
        

        #loss function that constrains Initial Conditions
        #ic_loss = map(lambda elem : (elem[0]-1)**2 if elem[1] < -1 else (elem[0] - (1-elem[1]))**2 if elem[1] < 1 else x[0]**2, zip(ic, x))
        ic_loss =(ic - torch.as_tensor(list(map(g, xinitial))))**2
        
       # test1=eq_loss
       ## test3=u_x
        #test2= ic_loss
        #test4=u*u_x

        return torch.mean(eq_loss)+2*resolution*torch.sum(ic_loss) #torch.mean(torch.cat((eq_loss, 2*ic_loss),dim=0)) #torch.mean(eq_loss.append() ) #

In [ ]:
pde      = InviscidBurgers()
pde_loss = InviscidBurgersLoss(pde)
loss_hist = []

t, x = generate_2Dgrid(0, 3, -2, 2, resolution)
optimizer = optim.Adam(pde.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=(0.1)**(1/5000))

################## Training and Plotting ##################
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 5));
font = {'size'   : 19}
plt.rc('font', **font)

p = None

for it in range(5000):
    try:
        optimizer.zero_grad()

        loss = pde_loss(t, x)
        loss_hist.append(loss.cpu().detach().numpy())

        loss.backward()
        optimizer.step()
        scheduler.step()
    
        if  it %100 == 0:
            t_plot, x_plot, u_plot = plot_form(x, t, pde(t, x), resolution)
            
            ax1.cla()
            ax1.set_xlabel('epoch')
            ax1.set_ylabel('loss')
            ax1.set_yscale('log')
            ax1.plot(loss_hist)
            
            ax2.cla()
            ax2.set_xlabel('x')
            ax2.set_ylabel('t')
            p = ax2.contourf(t_plot, x_plot, u_plot)
            
            display.display(plt.gcf())
            display.clear_output(wait=True)

    except KeyboardInterrupt:
        break

fig.colorbar(p)

In [ ]:
u = pde(torch.ones(50), torch.linspace(-2,2,50))
fig, (ax1) = plt.subplots(1,1, figsize = (12, 5));
ax1.plot(torch.linspace(-2,2,50), u.detach().numpy())

# ASIDE: MHD
<br>

In [ ]:
# The ODE solution will be modelled as a NN with 1 hidden layer of 40 nodes 
# and output 2 values
gamma = 2.0
Bx = 0.75
resolution = 100
class MHD(nn.Module):
    def __init__(self):
        super(MHD, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 40),
            nn.Tanh(),
            nn.Linear(40, 40),
            nn.Tanh(),
            nn.Linear(40, 7),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean = 0, std = 0.1)
                nn.init.constant_(m.bias, val = 0.0)
                
    def forward(self, t, x):
        tx = torch.stack((t,x), 1)
        #bdry = (abs(x) - 1).reshape(len(x), 1)
        return self.net(tx) #* bdry

# The loss function that will be used to train the NN will be constructed
# out of the DE
def initial_condition(x):
    if x < 0:
        return [1.0,0.0,0.0,0.0,1.0,0.0,1.0]
    else:
        return [0.125,0.0,0.0,0.0,-1.0,0.0,0.1]


def conserved(P):
    t_P = P.t()
    rho = t_P[0]
    vx = t_P[1]
    vy = t_P[2]
    vz = t_P[3]
    By = t_P[4]
    Bz = t_P[5]
    p = t_P[6]

    rhovx = vx*rho
    rhovy = vy*rho
    rhovz = vz*rho

    energy = (p/(gamma-1) + 1/2*(Bx**2+By**2+Bz**2) + rho/2*(vx**2+vy**2+vz**2))

    return torch.stack((rho, rhovx, rhovy, rhovz, By, Bz, energy)).t()
    #return torch.tensor(P)

def current(P):
    t_P = P.t()
    rho = t_P[0]
    vx = t_P[1]
    vy = t_P[2]
    vz = t_P[3]
    By = t_P[4]
    Bz = t_P[5]
    p = t_P[6]

    rhovx = vx*rho

    ps = p + 1/2*(Bx**2+By**2+Bz**2)
    energy = (p/(gamma-1) + 1/2*(Bx**2+By**2+Bz**2) + rho/2*(vx**2+vy**2+vz**2))

    return torch.stack((
        rhovx, 
        rhovx**2 + ps -Bx**2, 
        rhovx*vy - Bx*By, 
        rhovx*vz - Bx*Bz, 
        By*vx - Bx*vy, 
        Bz*vx - Bx*vz, 
        (energy+ps)*vx - Bx*(Bx*vx+By*vy+Bz*vz)
    )).t()

class MHDLoss(nn.Module):
    def __init__(self, approx_sol):
        super(MHDLoss, self).__init__()
        self.approx_sol = approx_sol
        self.conserved = conserved
        self.current = current
        
    def forward(self, t, x):

       # global test1, test2, test3, test4
      
        P = self.approx_sol(t, x)#.squeeze()
        U = self.conserved(P)
        J = self.current(P)

        eq = gradients(U, t) + gradients(J, x)

        # loss function associated with satisfying DE        
        eq_loss = (torch.flatten(eq))**2

        #define the initial values of t,x
        tinitial=torch.zeros(resolution)
        xinitial=torch.as_tensor([x[resolution*i] for i in range(resolution)])

        approx_sol_ic = self.approx_sol(tinitial,xinitial) #self.approx_sol(torch.zeros(x.detach().numpy().size), x)
        

        #loss function that constrains Initial Conditions
        #ic_loss = map(lambda elem : (elem[0]-1)**2 if elem[1] < -1 else (elem[0] - (1-elem[1]))**2 if elem[1] < 1 else x[0]**2, zip(ic, x))
        ic_loss =(approx_sol_ic - torch.as_tensor(list(map(initial_condition, xinitial))))**2

        return torch.mean(eq_loss)+2*resolution*torch.sum(ic_loss) #torch.mean(torch.cat((eq_loss, 2*ic_loss),dim=0)) #torch.mean(eq_loss.append() ) #

In [ ]:
pde      = MHD()
pde_loss = MHDLoss(pde)
loss_hist = []

t, x = generate_2Dgrid(0, 0.2, -1, 1, resolution)
optimizer = optim.Adam(pde.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=(0.1)**(1/5000))

################## Training and Plotting ##################
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 5));
font = {'size'   : 19}
plt.rc('font', **font)

p = None

for it in range(5000):
    try:
        optimizer.zero_grad()

        loss = pde_loss(t, x)
        loss_hist.append(loss.cpu().detach().numpy())

        loss.backward()
        optimizer.step()
        scheduler.step()
    
        if  it %100 == 0:
            t_plot, x_plot, u_plot = plot_form(x, t, pde(t, x).t()[0], resolution)
            
            ax1.cla()
            ax1.set_xlabel('epoch')
            ax1.set_ylabel('loss')
            ax1.set_yscale('log')
            ax1.plot(loss_hist)
            
            ax2.cla()
            ax2.set_xlabel('x')
            ax2.set_ylabel('t')
            p = ax2.contourf(t_plot, x_plot, u_plot)
            
            display.display(plt.gcf())
            display.clear_output(wait=True)

    except KeyboardInterrupt:
        break

fig.colorbar(p)
#hello

In [ ]:
u = pde(0.2*torch.ones(resolution), torch.linspace(-1,1,resolution))
fig, (ax1) = plt.subplots(1,1, figsize = (12, 5));
ax1.plot(torch.linspace(-2,2,resolution), u.t().detach().numpy()[2])

In [ ]:
u = pde(0.2*torch.zeros(resolution), torch.linspace(-1,1,resolution))
fig, (ax1) = plt.subplots(1,1, figsize = (12, 5));
ax1.plot(torch.linspace(-2,2,resolution), u.t().detach().numpy()[2])

In [ ]:
pde = MHD()
t = torch.ones(resolution, requires_grad=True)
x = torch.zeros(resolution, requires_grad=True)
gradients(conserved(pde(t, x)),t)
#output.grad
        

In [ ]:
t

In [ ]:
pp=pde(t,x).t()
trial=torch.einsum('i,j->ij',pp[0], pp[1])
trial.shape

In [ ]:
output

In [ ]:
pp=pde(t,x).t()
pp[0]

torch.stack((pp[0],pp[1]))

In [ ]:
torch.as_tensor(list(map(initial_condition, x))).shape

In [ ]:
torch.mean((pde(t, x) - torch.as_tensor(list(map(initial_condition, x))))**2)